# HTML tags position generation

Con la posición base del primer rectángulo, y aquellas posiciones de los saltos necesarios, calcularemos todos los restantes. Las posiciones vienen dadas por: horizontalizq, verticalarriba, horizontalder, verticalabajo

Input parameters: 
* gr_col: numero de 'columnas de grupos'
* gr_row: numero de 'filas de grupos'
* col: numero de columnas/grupo
* row: numero de filas/grupo 

In [6]:
import numpy as np

In [7]:
def mostrar(path):
    with open(path, 'r') as file:
        imgmap = file.read()
    print(imgmap)

## Creación de la clase

In [29]:
class mapeador():
    
    def __init__(self, path_input, path_output, gr_col, gr_row, col, row):
        self.path_input = path_input
        self.path_output = path_output
        self.gr_col = gr_col
        self.gr_row = gr_row
        self.col = col
        self.row = row
        
        self.importar_map()
        
    def importar_map(self):
        with open(self.path_input, 'r') as file:
            self.imgmap = file.read()
    
    def coordenadas(self):
        #Conseguimos el diccionario con las cordenadas del rectángulo inicial y los saltos necesarios
        coords_dict = {}
        split = [split.split('\" alt=\"') for split in self.imgmap.split('coords=\"')[1:]]
        
        for coords in split:
            alt = coords[1].split('\"  nohref')[0] #Careful: dos espacios entre \" y nohref
            coords_dict[int(alt)] = coords[0]
        
        return coords_dict
        
    def distancias(self):
        #Calculamos las distancias al rectángulo inicial (base)
        coords_dict = self.coordenadas()
        
        dist_dict = {}
        pos_base = [int(pos) for pos in coords_dict[1].split(",")]
        
        for entry in coords_dict.keys():
            if entry == 1:
                continue
            pos = [int(pos) for pos in coords_dict[entry].split(",")]
            dist_dict[entry] = np.subtract(pos,pos_base)
        
        #dist_good = {entry: list(dist_dict[entry][:2]) + list(dist_dict[entry][:2]) for entry in dist_dict.keys()} # Nos quedamos con las dos primeras entradas, y las repetimos.
        
        return dist_dict, pos_base
        
    def todas_posiciones(self):
        #obtenemos un diccionario alt <---> posiciones de los rectángulos
        dist_good, pos_base = self.distancias()
    
        alt = 1
        all_pos = {}
        all_pos[1] = np.array(pos_base)

        def calcular_posicion(alt,i,j,k,l):
            #Calculo del siguiente rectángulo.
            change = np.multiply(i,dist_good[2]) + np.multiply(j,dist_good[3]) + np.multiply(k,dist_good[col * row + 1])
            if l: 
                change += np.multiply(l,dist_good[col * row * gr_col + 1]) 

            pos = np.add(pos_base, change)
            alt += 1
            return alt, pos 

        for l in range(0,gr_row):
            for k in range(0,gr_col):
                for j in range(0,col):
                    for i in range(0,row):
                        if i == 0 and j == 0 and k == 0 and l == 0:
                            continue
                        alt, pos = calcular_posicion(alt,i,j,k,l)
                        all_pos[alt] = pos
        print(all_pos)
        
        return all_pos
    
    def preparar_output(self):
        #Eliminar los tags que ahora sustituiremos
        splitmap = self.imgmap.split('\n')
        finalmap = [] 
        for split in splitmap:
            if split.find('<area'):
                finalmap.append(split)

        return finalmap
    
    def generar_y_guardar(self):
        # Insertar los tags en la lista que irá al fichero
        all_pos = self.todas_posiciones()
        finalmap = self.preparar_output()
        
        for entry in all_pos.keys():
            intlist = all_pos[entry].tolist()
            strlist = [str(num) for num in intlist]
            val = ",".join(strlist)
            tag = f'<area shape="rect" coords="{val}" alt="{entry}"  nohref="nohref" />'
            finalmap.insert(-2,tag)
        
        with open(self.path_output, 'w') as file:
            file.write('\n'.join(finalmap))
        
        print("\nalright!")

### Caso 1: switch SX6536

In [30]:
#Detalles switch:
gr_col = 3
gr_row = 1
col = 3
row = 2

In [31]:
#Input/output:
path_input = 'imagenes_y_maps/[SX6536_detalle] (imported).map'
path_output = 'imagenes_y_maps/SX6536_detalle.map'

In [32]:
switchSX6536 = mapeador(path_input, path_output, gr_col, gr_row, col, row)

In [33]:
switchSX6536.generar_y_guardar()

{1: array([108,  61, 141,  76]), 2: array([108,  84, 142,  99]), 3: array([143,  61, 176,  76]), 4: array([143,  84, 177,  99]), 5: array([178,  61, 211,  76]), 6: array([178,  84, 212,  99]), 7: array([219,  61, 251,  76]), 8: array([219,  84, 252,  99]), 9: array([254,  61, 286,  76]), 10: array([254,  84, 287,  99]), 11: array([289,  61, 321,  76]), 12: array([289,  84, 322,  99]), 13: array([330,  61, 361,  76]), 14: array([330,  84, 362,  99]), 15: array([365,  61, 396,  76]), 16: array([365,  84, 397,  99]), 17: array([400,  61, 431,  76]), 18: array([400,  84, 432,  99])}

alright!


### Caso 2: switch SX6036

In [34]:
#Detalles switch:
gr_col = 3
gr_row = 1
col = 6
row = 2

In [35]:
#Input/output:
path_input = 'imagenes_y_maps/[StorageReview-Mellanox-SX6036-InfiniBand-Switch-Front] (imported).map'
path_output = 'imagenes_y_maps/SX6036_IB_Front.map'

In [36]:
switchSX6536 = mapeador(path_input, path_output, gr_col, gr_row, col, row)
switchSX6536.generar_y_guardar()

{1: array([ 70,  51, 105,  67]), 2: array([ 71,  76, 106,  92]), 3: array([108,  51, 142,  67]), 4: array([109,  76, 143,  92]), 5: array([146,  51, 179,  67]), 6: array([147,  76, 180,  92]), 7: array([184,  51, 216,  67]), 8: array([185,  76, 217,  92]), 9: array([222,  51, 253,  67]), 10: array([223,  76, 254,  92]), 11: array([260,  51, 290,  67]), 12: array([261,  76, 291,  92]), 13: array([300,  53, 334,  67]), 14: array([301,  78, 335,  92]), 15: array([338,  53, 371,  67]), 16: array([339,  78, 372,  92]), 17: array([376,  53, 408,  67]), 18: array([377,  78, 409,  92]), 19: array([414,  53, 445,  67]), 20: array([415,  78, 446,  92]), 21: array([452,  53, 482,  67]), 22: array([453,  78, 483,  92]), 23: array([490,  53, 519,  67]), 24: array([491,  78, 520,  92]), 25: array([530,  55, 563,  67]), 26: array([531,  80, 564,  92]), 27: array([568,  55, 600,  67]), 28: array([569,  80, 601,  92]), 29: array([606,  55, 637,  67]), 30: array([607,  80, 638,  92]), 31: array([644,  5